In [1]:
import numpy as np
import tensorflow as tf
tf.__version__

/Users/georgymh/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'1.6.0'

In [2]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
X_train = np.concatenate((mnist.train.images, mnist.validation.images))
y_train = np.concatenate((
            np.asarray(mnist.train.labels, dtype=np.int32),
            np.asarray(mnist.validation.labels, dtype=np.int32),
          ))
X_test = mnist.test.images
y_test = np.asarray(mnist.test.labels, dtype=np.int32)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [3]:
X_train_ = X_train.reshape(-1, 784)
y_train_ = np.eye(10)[y_train]
X_test_ = X_test.reshape(-1, 784)
y_test_ = np.eye(10)[y_test]
X_train_.shape, y_train_.shape

((60000, 784), (60000, 10))

In [67]:
class Perceptron:
    def __init__(self, hparams):
        self.n_input = 784
        self.n_classes = 10
        self.n_hidden1 = 200
        self.n_hidden2 = 200
        self.learning_rate = hparams['learning_rate']
        
    def preprocess_input(self, x):
        return x
    
    def build_model(self, input_layer):
        with tf.variable_scope("layer1"):
            self.layer1 = tf.layers.dense(input_layer, self.n_hidden1, tf.nn.relu)
        with tf.variable_scope("layer2"):    
            self.layer2 = tf.layers.dense(self.layer1, self.n_hidden2, tf.nn.relu)
        with tf.variable_scope("logits_layer"):
            self.logits = tf.layers.dense(self.layer2, self.n_classes)
            
    def build_loss(self):
        self.loss = tf.losses.sparse_softmax_cross_entropy(
            labels=self.labels, 
            logits=self.logits,
            loss_collection=tf.GraphKeys.LOSSES
        )
        
    def build_optimizer(self):
        optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
        self.optimizer = optimizer.minimize(
            loss=self.loss, 
            global_step=tf.train.get_global_step(),
            name='train_op'
        )

    def build_predictions_obj(self):
        logits = self.logits
        classes = tf.argmax(input=self.logits, axis=1, name="classes_tensor")
        probabilities = tf.nn.softmax(self.logits, name="softmax_tensor")
        
        tf.add_to_collection('predictions', logits)
        tf.add_to_collection('predictions', classes)
        tf.add_to_collection('predictions', probabilities)
        
        self.predictions = {
            "logits": logits,
            "classes": classes,
            "probabilities": probabilities
        }
        
    def build_eval_metric(self):
        self.eval_metric_ops = {
          "accuracy": tf.metrics.accuracy(labels=self.labels, predictions=self.predictions["classes"])
        }
        
    def build_all(self):
        self.build_model()
        self.build_loss()
        self.build_optimizer()
        self.build_predictions_obj()
        self.build_eval_metric()
            
    def get_estimator(self, mode):
        estimator = None
        self.build_predictions_obj()
        if mode == tf.estimator.ModeKeys.PREDICT:
            estimator = tf.estimator.EstimatorSpec(mode=mode, predictions=self.predictions)
        elif mode == tf.estimator.ModeKeys.TRAIN:
            self.build_loss()
            self.build_optimizer()
            estimator = tf.estimator.EstimatorSpec(mode=mode, loss=self.loss, train_op=self.optimizer)
        elif mode == tf.estimator.ModeKeys.EVAL:
            self.build_eval_metric()
            self.build_loss()
            estimator = tf.estimator.EstimatorSpec(mode=mode, loss=self.loss, eval_metric_ops=self.eval_metric_ops)
        return estimator
    
    def get_model(self, features, labels, mode):
        """
        When using the Estimator API, features will come as a TF Tensor already.
        """
        # Do pre-processing if necessary.
        self.input_layer = self.preprocess_input(features["x"])
        self.labels = labels

        # Define the model.
        self.build_model(self.input_layer)
        
        # Build and return the estimator.
        return self.get_estimator(mode)

In [79]:
# Create the Estimator
hparams = {'learning_rate': 0.001}
model = Perceptron(hparams)
classifier = tf.estimator.Estimator(model_fn=model.get_model, model_dir="./mnist/")

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=1)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './mnist/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x124a2a898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [19]:
"""
ESTIMATOR WAY
1- Server sends its 'input_fn()' defining the model and initial weights to clients 
    [yes]
2- Clients create an estimator with the weights, train on their data, and send back the new weights to the server 
    [yes -- put weights in folder, create estimator with checkpoint, run fit on estimator, sends the latest checkpoint]
3- As the server receives the new weights, it averages the weights; once done, it evaluates the model on a test set 
    [yes -- as weights are received we average them; once done, we load the model, assign the weights, save the model,
    under new name, and run an eval on the test set]


TRADITIONAL WAY

"""

tf.reset_default_graph()

# CLIENT
def train_model(config):
    """
    Loads the model and weights found in server-checkpoints/ (if found), trains using the hyperparameters 
    in config, then returns a list of new checkpoints after training. 
    """
    def make_weights_dict(sess, collection):
        return {tensor.name:sess.run(tensor) for tensor in collection}
    
    graph = tf.Graph()
    with tf.Session(graph=graph) as sess:
        #new_saver = tf.train.import_meta_graph(metagraph)
        new_saver = tf.train.import_meta_graph(tf.train.latest_checkpoint('./mnist/') + '.meta')
        try:
            new_saver.restore(sess, tf.train.latest_checkpoint('./mnist/'))
        except e:
            # Couldn't load model.
            print("Error...")
            raise e 
        
        input = graph.get_collection('input_tensor:0')
        print(input)
        
        logits, classes, probabilities = graph.get_collection('predictions')
        print(logits)
        
        collection = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
        print(make_weights_dict(sess, collection))
        
        for i in range(20000):
            batch = mnist.train.next_batch(50)
            train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
            if i % 100 == 0:
                train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
                print('step %d, training accuracy %g' % (i, train_accuracy))
            if i % 1000 == 0:
                # test accuracy
            

        print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

# SERVER
def aggregate_models(config):
    """
    Loads the models found in updates/
    """
    

train_model(None)

INFO:tensorflow:Restoring parameters from ./mnist/model.ckpt-23613
[]
Tensor("logits_layer/dense/BiasAdd:0", shape=(100, 10), dtype=float32)
{'layer1/dense/kernel:0': array([[-0.01104958, -0.03149257,  0.0609542 , ...,  0.04887503,
        -0.05664973,  0.00778133],
       [-0.03988727, -0.0560302 , -0.0709151 , ..., -0.02928087,
         0.05130166, -0.07581948],
       [ 0.05992617,  0.02358893,  0.01766432, ...,  0.03194337,
        -0.04137886, -0.02968797],
       ...,
       [-0.00962557, -0.06150879, -0.05601227, ..., -0.03184977,
        -0.02179823,  0.03371893],
       [ 0.05295402, -0.04876651,  0.07415543, ...,  0.05580209,
         0.04050494, -0.06941249],
       [-0.05708542,  0.05524662, -0.03389598, ...,  0.03212387,
        -0.06944697,  0.03257002]], dtype=float32), 'layer1/dense/bias:0': array([-0.03311902, -0.14471869, -0.1002543 , -0.02374201, -0.01721681,
       -0.04443753,  0.04729019, -0.1204375 ,  0.06492611,  0.07457326,
       -0.13051437,  0.20195019, -0.0

In [22]:
# import the inspect_checkpoint library
from tensorflow.python.tools import inspect_checkpoint as chkp

# print all tensors in checkpoint file
chkp.print_tensors_in_checkpoint_file(tf.train.latest_checkpoint('./mnist/'), tensor_name='', all_tensors=True)

tensor_name:  beta1_power
0.0
tensor_name:  beta2_power
5.4900008e-11
tensor_name:  global_step
23613
tensor_name:  layer1/dense/bias
[-0.03311902 -0.14471869 -0.1002543  -0.02374201 -0.01721681 -0.04443753
  0.04729019 -0.1204375   0.06492611  0.07457326 -0.13051437  0.20195019
 -0.00694241 -0.05435921 -0.0615164  -0.281682   -0.06439892 -0.12723564
 -0.12199339  0.07591756  0.15622403  0.00292117  0.05217758  0.07655474
 -0.00087063 -0.09814455 -0.021318    0.07600338 -0.12191084  0.0075237
  0.015504   -0.11277445  0.01906838  0.15454742 -0.2501048   0.05295547
  0.13423464 -0.0060253   0.03524339  0.03554297 -0.00233773 -0.08682412
 -0.19070244 -0.23528357  0.01556841  0.02184211  0.06357921  0.1024883
 -0.14207996 -0.0098196   0.24104531  0.06414191  0.01587448 -0.07740874
  0.01679966 -0.205904    0.09572794  0.12011979 -0.03666602 -0.1607297
  0.03449899 -0.07151773 -0.14488713  0.04340551  0.07608213  0.10185873
 -0.06036101 -0.0442534   0.07638665 -0.03058627  0.01391575  0.17

In [81]:
%%time
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train_},
    y=y_train,
    batch_size=1,
    num_epochs=None,
    shuffle=False
)
classifier.train(input_fn=train_input_fn, steps=1, hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./mnist/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2 into ./mnist/model.ckpt.
INFO:tensorflow:probabilities = [[0.04025666 0.10458057 0.10845653 0.10694748 0.08592811 0.0421818
  0.06704649 0.21724518 0.07609104 0.15126604]]
INFO:tensorflow:loss = 1.5267287, step = 2
INFO:tensorflow:Loss for final step: 1.5267287.
CPU times: user 719 ms, sys: 33.5 ms, total: 752 ms
Wall time: 739 ms


In [7]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test_},
    y=y_test,
    num_epochs=1,
    shuffle=False
)
eval_results = classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Starting evaluation at 2018-03-29-01:28:58
INFO:tensorflow:Restoring parameters from mnist/model.ckpt-23613
INFO:tensorflow:Finished evaluation at 2018-03-29-01:28:59
INFO:tensorflow:Saving dict for global step 23613: accuracy = 0.9832, global_step = 23613, loss = 0.1050892
{'accuracy': 0.9832, 'loss': 0.1050892, 'global_step': 23613}


In [60]:
def import_weights_as_np():
    tf.reset_default_graph()
    weights = []
    with tf.Session().as_default() as sess:
        new_saver = tf.train.import_meta_graph(tf.train.latest_checkpoint('./mnist/') + '.meta')
        new_saver.restore(sess, tf.train.latest_checkpoint('./mnist/'))
        collection = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
        weights = {tensor.name:sess.run(tensor) for tensor in collection}
    return weights
        
weights = export_weights_as_np(Perceptron, hparams)
weights2 = export_weights_as_np(Perceptron, hparams)
sum(w.size for _, w in weights.items()), weights['layer1/dense/bias:0'][:5]

INFO:tensorflow:Restoring parameters from ./mnist/model.ckpt-23613
INFO:tensorflow:Restoring parameters from ./mnist/model.ckpt-23613


(199210,
 array([-0.03311902, -0.14471869, -0.1002543 , -0.02374201, -0.01721681],
       dtype=float32))

In [62]:
new_weights = {}
for key1, key2 in zip(sorted(weights.keys()), sorted(weights2.keys())):
    assert key1 == key2, 'Error with keys'
    new_weights[key1] = weights[key1] + weights2[key2]
new_weights['layer1/dense/bias:0'][:5]

array([-0.06623803, -0.28943738, -0.20050861, -0.04748401, -0.03443361],
      dtype=float32)

In [58]:
def load_weights(weights, round_num):
    tf.reset_default_graph()
    with tf.Session().as_default() as sess:
        new_saver = tf.train.import_meta_graph(tf.train.latest_checkpoint('./mnist/') + '.meta')
        new_saver.restore(sess, tf.train.latest_checkpoint('./mnist/')) # to load non-trainable variables 
        
        collection = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
        for tensor in collection:
            assign_op = tensor.assign(weights[tensor.name])
            sess.run(assign_op)
        
        save_path = new_saver.save(sess, "./mnist/model.ckpt", global_step=round_num)
        print("Model saved in path: %s" % save_path)
load_weights(new_weights, 1337)

INFO:tensorflow:Restoring parameters from ./mnist/model.ckpt
Model saved in path: ./mnist/model.ckpt-1337


In [70]:
from google.protobuf import json_format

def export_metagraph_as_json(model, hparams):
    tf.reset_default_graph()
    with tf.Graph().as_default() as graph:
        m = model(hparams)
        m.build_model(tf.placeholder(tf.float32, (None, 28*28)))
        m.build_predictions_obj()
        graph_def = graph.as_graph_def()
        json_string = json_format.MessageToJson(graph_def)
    return json_string

export_metagraph_as_json(Perceptron, hparams)[:50]

'{\n  "node": [\n    {\n      "name": "Placeholder",\n '

In [75]:
def export_metagraph():
    tf.reset_default_graph()
    with tf.Session().as_default() as sess:
        new_saver = tf.train.import_meta_graph(tf.train.latest_checkpoint('./mnist/') + '.meta')
#         m = model(hparams)
#         m.build_model(tf.placeholder(tf.float32, (None, 28*28), name='input_tensor'))
#         m.build_predictions_obj()
        meta_graph_def = tf.train.export_meta_graph()
    return meta_graph_def

#mgraph = export_metagraph()
#metagraph = export_metagraph(Perceptron, hparams)
tf.train.latest_checkpoint('./mnist/')

'./mnist/model.ckpt-23613'

In [27]:
def build_latest_graph():
    tf.reset_default_graph()
    loaded_graph = tf.Graph()
    with tf.Session(graph=loaded_graph) as sess:
        #new_saver = tf.train.import_meta_graph(metagraph)
        new_saver = tf.train.import_meta_graph(tf.train.latest_checkpoint('./mnist/') + '.meta')
        new_saver.restore(sess, tf.train.latest_checkpoint('./mnist/'))
        
        print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))
        
        input = loaded_graph.get_tensor_by_name('enqueue_input/Placeholder_1:0')
        #input = loaded_graph.get_collection('enqueue_input')
        #input = loaded_graph.get_collection('input_tensor:0')
        print(input)
        
        logits, classes, probabilities = loaded_graph.get_collection('predictions')
        print(logits)
        
#         def get_model(features, labels, mode):
#             return tf.estimator.EstimatorSpec(mode=mode, predictions=classes)
            
        
#         hparams = {'learning_rate': 0.001}
#         model = Perceptron(hparams)
#         classifier = tf.estimator.Estimator(model_fn=get_model, model_dir="mnist2/")
#         # Evaluate the model and print results
#         eval_input_fn = tf.estimator.inputs.numpy_input_fn(
#             x={"x": X_test_},
#             y=y_test,
#             num_epochs=1,
#             shuffle=False
#         )
#         eval_results = classifier.evaluate(input_fn=eval_input_fn)
#         print(eval_results)
        
        
        
#         pred = sess.run(classes, feed_dict={input: X_test_[:10]})
#         print(pred, y_test[:10])
        
        #print(sess.run(predictions[0]))
    #print(sess.run(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)))
    #   for step in xrange(1000000):
    #     sess.run(train_op)

build_latest_graph()

INFO:tensorflow:Restoring parameters from ./mnist/model.ckpt-23613
[<tf.Variable 'global_step:0' shape=() dtype=int64_ref>, <tf.Variable 'layer1/dense/kernel:0' shape=(784, 200) dtype=float32_ref>, <tf.Variable 'layer1/dense/bias:0' shape=(200,) dtype=float32_ref>, <tf.Variable 'layer2/dense/kernel:0' shape=(200, 200) dtype=float32_ref>, <tf.Variable 'layer2/dense/bias:0' shape=(200,) dtype=float32_ref>, <tf.Variable 'logits_layer/dense/kernel:0' shape=(200, 10) dtype=float32_ref>, <tf.Variable 'logits_layer/dense/bias:0' shape=(10,) dtype=float32_ref>, <tf.Variable 'beta1_power:0' shape=() dtype=float32_ref>, <tf.Variable 'beta2_power:0' shape=() dtype=float32_ref>, <tf.Variable 'layer1/dense/kernel/Adam:0' shape=(784, 200) dtype=float32_ref>, <tf.Variable 'layer1/dense/kernel/Adam_1:0' shape=(784, 200) dtype=float32_ref>, <tf.Variable 'layer1/dense/bias/Adam:0' shape=(200,) dtype=float32_ref>, <tf.Variable 'layer1/dense/bias/Adam_1:0' shape=(200,) dtype=float32_ref>, <tf.Variable 'la

In [ ]:
tf.reset_default_graph()

m = Perceptron(hparams)
m.build_model(tf.placeholder(tf.float32, (None, 28*28)))

weights = []
saver = tf.train.Saver()
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  #new_saver = tf.train.import_meta_graph('mnist/model.ckpt-23605.meta')
  #new_saver.restore(sess, tf.train.latest_checkpoint('mnist'))
  #saver.restore(sess, "./mnist/model.ckpt") 
    
  ckpt = tf.train.get_checkpoint_state("./mnist/model.ckpt-23604")
  print(ckpt)
  
    
  print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))
  weights = sess.run(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))
weights

In [68]:
def prepare_for_training():
    tf.reset_default_graph()
    with tf.Session().as_default() as sess:
        m = Perceptron(hparams)
        m.build_all()
        tf.train.import_meta_graph(filename='./mnist/lol.meta')
    tf.reset_default_graph()

prepare_for_training()

TypeError: build_model() missing 1 required positional argument: 'input_layer'

In [82]:
tf.train.latest_checkpoint('./mnist')

'./mnist/model.ckpt-2'